In [95]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor


from pathlib import Path
from typing import Tuple

In [96]:
def _CYME(df: pd.DataFrame) -> float:
    """ Compute the CYME metric, that is 1/2(median(yearly error) + median(monthly error))"""

    yearly_agg = df.groupby("cluster_nl")[["target", "prediction"]].sum().reset_index()
    yearly_error = abs((yearly_agg["target"] - yearly_agg["prediction"])/yearly_agg["target"]).median()

    monthly_error = abs((df["target"] - df["prediction"])/df["target"]).median()

    return 1/2*(yearly_error + monthly_error)


def _metric(df: pd.DataFrame) -> float:
    """Compute metric of submission.

    :param df: Dataframe with target and 'prediction', and identifiers.
    :return: Performance metric
    """
    df = df.copy()
    df["date"] = pd.to_datetime(df["date"])

    # Split 0 actuals - rest
    zeros = df[df["zero_actuals"] == 1]
    recent = df[df["zero_actuals"] == 0]

    # weight for each group
    zeros_weight = len(zeros)/len(df)
    recent_weight = 1 - zeros_weight

    # Compute CYME for each group
    return round(recent_weight*_CYME(recent) + zeros_weight*min(1,_CYME(zeros)),8)


def compute_metric(submission: pd.DataFrame) -> Tuple[float, float]:
    """Compute metric.

    :param submission: Prediction. Requires columns: ['cluster_nl', 'date', 'target', 'prediction']
    :return: Performance metric.
    """

    submission["date"] = pd.to_datetime(submission["date"])
    submission = submission[['cluster_nl', 'date', 'target', 'prediction', 'zero_actuals']]

    return _metric(submission)

In [97]:
# Define file paths

features_cols = [
    "brand",
    "che_pc_usd",
    "che_perc_gdp",
    "corporation",
    "country",
    "launch_date",
    "ind_launch_date",
    "indication",
    "insurance_perc_che",
    "population",
    "prev_perc",
    "price_month",
    "price_unit",
    "public_perc_che",
    "therapeutic_area",
    "Country_Group",
    "Price_Group",
    "indication_number",
    "avg_price_per_year",
    "month_number"
]
target_col = "target"
id_col = ["date","cluster_nl"]

base_dir = os.path.join(os.path.dirname(os.getcwd()), "dataset")
therap = os.path.join(os.path.dirname(os.getcwd()), "dataset", "therapeutic_area")
# Load datasets
# data = pd.read_csv(f"{base_dir}/train_data.csv", usecols=features_cols + [target_col] + id_col)
data = {}
y = {}
filenames = os.listdir(therap)

therapeutic_areas = []
extracted_parts = []
for filename in filenames:
    if filename.startswith('subset'):
        continue
    filename_no_ext = filename.split('.')[0]
    therapeutic_areas.append(filename_no_ext)
    # Load the dataset
    filepath = f"{therap}/{filename}"
    df = pd.read_csv(filepath, usecols=features_cols + [target_col] + id_col)
    print(f"Data loaded for therapeutic areas: {filename}")
    data[filename_no_ext] = df
    y[filename_no_ext] = df[target_col]
    extracted_parts.append(filename_no_ext)


test_data = pd.read_csv(f"{base_dir}/submission_data_TRY2.csv", usecols=features_cols + id_col)

Data loaded for therapeutic areas: 032C_051D_644A_66C5_6CEE.csv
Data loaded for therapeutic areas: 22ED.csv
Data loaded for therapeutic areas: 96D7.csv
Data loaded for therapeutic areas: CD59.csv
Data loaded for therapeutic areas: 4BA5_645F_8E53_980E.csv


In [98]:
numeric_features = {}
categorical_features = {}
for i in extracted_parts:
    # convert int64 to float64
    data[i] = data[i].astype({"Country_Group": "float64"})
    data[i] = data[i].astype({"Price_Group": "float64"})
    data[i] = data[i].astype({"indication_number": "float64"})
    numeric_features[i] = data[i].select_dtypes(include=['float64']).drop(columns=[target_col], errors='ignore').columns
    categorical_features[i] = data[i].select_dtypes(include=['object']).columns

test_data = test_data.astype({"Country_Group": "float64"})
test_data = test_data.astype({"Price_Group": "float64"})
test_data = test_data.astype({"indication_number": "float64"})

# Separate numeric and categorical features for imputation


In [99]:
X = {}
for i in extracted_parts:
    # Drop unnecessary columns
    X[i] = data[i].drop(columns=[target_col]+["cluster_nl"])

X_test = test_data.drop(columns=["cluster_nl"])

# Preprocessing pipeline
def preprocess_data(X, preprocessor=None, fit=True):
    numerical_features = X.select_dtypes(include=['float64']).columns
    categorical_features = X.select_dtypes(include=['category']).columns

    if preprocessor is None:
        numerical_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ])
        categorical_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
        ])

        preprocessor = ColumnTransformer(
            transformers=[
                ('num', numerical_transformer, numerical_features),
                ('cat', categorical_transformer, categorical_features)
            ]
        )

    if fit:
        X_transformed = preprocessor.fit_transform(X)
    else:
        X_transformed = preprocessor.transform(X)

    X_transformed = np.array(X_transformed)

    return X_transformed, preprocessor

# Preprocess data
X_transformed = {}
for i in extracted_parts:
    X_transformed[i], preprocessor = preprocess_data(X[i], fit=True)


X_test_transformed = preprocess_data(X_test, preprocessor=preprocessor, fit=False)

In [100]:
X_train = {}
X_valid = {}
y_train = {}
y_valid = {}
for i in extracted_parts:
    print(f"X_transformed shape for {i}: {X_transformed[i].shape}")
    X_train[i], X_valid[i], y_train[i], y_valid[i] = train_test_split(X_transformed[i], y[i], test_size=0.05, random_state=42)


X_transformed shape for 032C_051D_644A_66C5_6CEE: (44331, 12)
X_transformed shape for 22ED: (112, 11)
X_transformed shape for 96D7: (45858, 12)
X_transformed shape for CD59: (3992, 12)
X_transformed shape for 4BA5_645F_8E53_980E: (24150, 12)


In [101]:
import lightgbm as lgb
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

models = {}
for i in extracted_parts:
    models[f'XGBoost-{i}'] = XGBRegressor(n_estimators=100, random_state=42)
    models[f'LightGBM-{i}'] = lgb.LGBMRegressor(n_estimators=100, random_state=42)
    models[f'RandomForest10-{i}'] = RandomForestRegressor(n_estimators=10, random_state=42)
    models[f'RandomForest100-{i}'] = RandomForestRegressor(n_estimators=100, random_state=42)
    models[f'RandomForest200-{i}'] = RandomForestRegressor(n_estimators=200, random_state=42)

print(models)
results = {}

print(X_train.keys())
for name, model in models.items():
    i = name.split('-')[1]
    model.fit(X_train[i], y_train[i])
    scores = cross_val_score(model, X_train[i], y_train[i], scoring='neg_root_mean_squared_error', cv=5)
    results[name] = -scores.mean()


{'XGBoost-032C_051D_644A_66C5_6CEE': XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...), 'LightGBM-032C_051D_644A_66C5_6CEE': LGBMRegressor(random_state=42), 'RandomForest10-032C_051D_644A_66C5_6CEE': RandomForestRegressor(n_estimators=10, random_state=42), 'RandomForest100-032C_051D_644A_66C5_6CEE': Rando

In [102]:
best_model = {}
best_results = {}

for i in extracted_parts:
    best_model[i] = None
    # get name of models
    name_models = [name.split("-")[0] for name in results.keys() if name.endswith(i)]
    print(name_models)
    # get best model for each therapeutic area by name
    best_model[i] = min(name_models, key=lambda x: results[x + "-" + i])
    print(f"Best model for {i}: {best_model[i]}, RMSE: {results[best_model[i] + '-' + i]}")
    best_results[best_model[i] + '-' + i] = results[best_model[i] + '-' + i]
    # remove other models for this therapeutic area
    for name in name_models:
        if name != best_model[i]:
            del models[name + "-" + i]
    
best_results
# make it a dataframe
best_results_df = pd.DataFrame(best_results.items(), columns=['model', 'RMSE'])
best_results_df

['XGBoost', 'LightGBM', 'RandomForest10', 'RandomForest100', 'RandomForest200']
Best model for 032C_051D_644A_66C5_6CEE: RandomForest200, RMSE: 0.05729333976322774
['XGBoost', 'LightGBM', 'RandomForest10', 'RandomForest100', 'RandomForest200']
Best model for 22ED: LightGBM, RMSE: 0.003169730670502425
['XGBoost', 'LightGBM', 'RandomForest10', 'RandomForest100', 'RandomForest200']
Best model for 96D7: RandomForest200, RMSE: 0.0842140886306147
['XGBoost', 'LightGBM', 'RandomForest10', 'RandomForest100', 'RandomForest200']
Best model for CD59: LightGBM, RMSE: 0.028369830234089365
['XGBoost', 'LightGBM', 'RandomForest10', 'RandomForest100', 'RandomForest200']
Best model for 4BA5_645F_8E53_980E: RandomForest200, RMSE: 0.032116618837708576


,model,RMSE
0,RandomForest200-032C_051D_644A_66C5_6CEE,0.057293
1,LightGBM-22ED,0.003170
2,RandomForest200-96D7,0.084214
3,LightGBM-CD59,0.028370
4,RandomForest200-4BA5_645F_8E53_980E,0.032117


In [103]:
print(results)
# Display RMSE results
results_df = pd.DataFrame(results.items(), columns=['Model', 'RMSE']).sort_values(by='RMSE')
results_df['RMSE'] = results_df['RMSE'].map("{:.5f}".format)
results_df

{'XGBoost-032C_051D_644A_66C5_6CEE': 0.06434076558573723, 'LightGBM-032C_051D_644A_66C5_6CEE': 0.084598354387325, 'RandomForest10-032C_051D_644A_66C5_6CEE': 0.05829108571653796, 'RandomForest100-032C_051D_644A_66C5_6CEE': 0.05738093162963418, 'RandomForest200-032C_051D_644A_66C5_6CEE': 0.05729333976322774, 'XGBoost-22ED': 0.0033219969480492383, 'LightGBM-22ED': 0.003169730670502425, 'RandomForest10-22ED': 0.0034265944511729053, 'RandomForest100-22ED': 0.0032500887415999316, 'RandomForest200-22ED': 0.003271137275466413, 'XGBoost-96D7': 0.09260407369100732, 'LightGBM-96D7': 0.12526872983637766, 'RandomForest10-96D7': 0.08702616529312196, 'RandomForest100-96D7': 0.08439171258172976, 'RandomForest200-96D7': 0.0842140886306147, 'XGBoost-CD59': 0.02876271964862446, 'LightGBM-CD59': 0.028369830234089365, 'RandomForest10-CD59': 0.029892845741439616, 'RandomForest100-CD59': 0.029184511429106698, 'RandomForest200-CD59': 0.029008810301620808, 'XGBoost-4BA5_645F_8E53_980E': 0.03296415181660079, 'L

,Model,RMSE
6,LightGBM-22ED,0.00317
8,RandomForest100-22ED,0.00325
9,RandomForest200-22ED,0.00327
5,XGBoost-22ED,0.00332
7,RandomForest10-22ED,0.00343
16,LightGBM-CD59,0.02837
15,XGBoost-CD59,0.02876
19,RandomForest200-CD59,0.02901
18,RandomForest100-CD59,0.02918
17,RandomForest10-CD59,0.02989


In [104]:
X_valid_df = {}
for i in extracted_parts:
    X_valid_df[i] = pd.DataFrame(X_valid[i])

# Predict on validation set
predictions = {}
for i in extracted_parts:
    # get key of best model
    key = best_model[i] + "-" + i
    predictions[i] = models[key].predict(X_valid[i])

# Compute metric
metrics = {}
for i in extracted_parts:
    # create an column with zeros

    data[i]["zero_actuals"] = [0]*len(data[i]["cluster_nl"])
    print(data[i]["zero_actuals"])
    metrics[i] = compute_metric(pd.concat([data[i]["cluster_nl"], data[i]["date"], pd.Series(y_valid[i]), pd.Series(predictions[i], name="prediction"), data[i]["zero_actuals"]], axis=1))


metrics

0        0
1        0
2        0
3        0
4        0
        ..
44326    0
44327    0
44328    0
44329    0
44330    0
Name: zero_actuals, Length: 44331, dtype: int64
0      0
1      0
2      0
3      0
4      0
      ..
107    0
108    0
109    0
110    0
111    0
Name: zero_actuals, Length: 112, dtype: int64
0        0
1        0
2        0
3        0
4        0
        ..
45853    0
45854    0
45855    0
45856    0
45857    0
Name: zero_actuals, Length: 45858, dtype: int64
0       0
1       0
2       0
3       0
4       0
       ..
3987    0
3988    0
3989    0
3990    0
3991    0
Name: zero_actuals, Length: 3992, dtype: int64
0        0
1        0
2        0
3        0
4        0
        ..
24145    0
24146    0
24147    0
24148    0
24149    0
Name: zero_actuals, Length: 24150, dtype: int64


{'032C_051D_644A_66C5_6CEE': 0.55407257,
 '22ED': 0.19541204,
 '96D7': 0.49325772,
 'CD59': 0.26617961,
 '4BA5_645F_8E53_980E': 0.56156417}

In [105]:
X_test_transformed = preprocess_data(X_test, preprocessor=preprocessor, fit=False)
print(X_test_transformed[0])

# Ensure indices align between test_data and X_test_transformed
X_test_transformed = pd.DataFrame(X_test_transformed[0], index=test_data.index)  # Specify column names if necessary

def extract_part(text):
    return text.split('_')[2].split('.')[0]

X_test_transformed['therapeutic_area'] = test_data['therapeutic_area'].apply(extract_part)
print(X_test_transformed)

# Initialize a DataFrame to store the results
results_final = pd.DataFrame()

# Iterate over each therapeutic area
for area in models.keys():
    keyy = area.split('-')[1].split('_')
    print(keyy)
    # Filter test data for the current therapeutic area
    area_indices = X_test_transformed[X_test_transformed['therapeutic_area'].isin(keyy)].index
    X_area = X_test_transformed.loc[area_indices].drop('therapeutic_area', axis=1)

    # Get the corresponding model
    model = models[area]
    print(models)

    # Make predictions
    if not X_area.empty:
      predictions = model.predict(X_area)

      # Prepare the result DataFrame
      test_area = test_data.loc[area_indices]
      area_results = pd.DataFrame({
          'date': pd.to_datetime(test_area['date']).dt.strftime("%m/%d/%Y"),
          'cluster_nl': test_area['cluster_nl'],
          'prediction': predictions,        
      })

      # Append to the results
      results_final = pd.concat([results_final, area_results], ignore_index=True)

results_final.to_csv('result.csv', index=False)
print("Result file saved as 'result.csv'")

[[ 1.35633647 -0.66060897 -0.2479499  ...  1.         -0.546629
  -0.53176886]
 [ 1.35633647 -0.66060897 -0.2479499  ...  1.         -0.546629
   0.5320094 ]
 [-1.35273906 -1.85085513  0.06090513 ...  1.          0.80739842
  -0.71592336]
 ...
 [-0.92194009 -0.5761904   0.86290101 ...  1.         -0.546629
   8.21231286]
 [-0.92194009 -0.5761904   0.86290101 ...  1.         -0.546629
   7.60051463]
 [-1.3511863  -1.78526217 -0.14335483 ...  1.         -0.546629
  10.3647777 ]]
             0         1         2         3          4          5          6  \
0     1.356336 -0.660609 -0.247950 -0.891348  -0.465215  -0.198779  -0.575722   
1     1.356336 -0.660609 -0.247950 -0.891348   1.178832 -26.689030   0.844140   
2    -1.352739 -1.850855  0.060905  5.701652  -0.359829 -26.689030  -0.666875   
3    -0.913969 -0.503665  0.822033 -0.779632  -1.257814 -26.689030   2.782282   
4    -1.351186 -1.785262 -0.143355  1.691634  -1.319395 -26.689030  13.182867   
...        ...       ...       .